# Connect Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir /content/drive/MyDrive/colab
%cd /content/drive/MyDrive/colab


# ***1. Load Dataset - EDA***

Dataset
- https://mmlab.ie.cuhk.edu.hk/projects/CelebA.html (lỗi bboxes)
- https://datasets.activeloop.ai/docs/ml/datasets/celeba-dataset/
- https://www.kaggle.com/datasets/jessicali9530/celeba-dataset  (lỗi bboxes)


In [ ]:
!pip install deeplake


In [ ]:
import deeplake
ds_train = deeplake.load("hub://activeloop/celeb-a-train")[:10]

img_list_train = ds_train['images'],
anno_list_train = ds_train['boxes']
print(img_list_train)
print(anno_list_train)

import numpy as np
import matplotlib.pyplot as plt
# Xem qua một số hình ảnh từ tập huấn luyện
i=0
image = ds_train['images'][i]  # Tensor(key='images', index=Index([0]))
print(image)
# In hình dạng và kiểu dữ liệu của ảnh
print("Shape of the image:", image.shape) # Shape of the image: (295, 285, 3)
print("Data type of the image:", image.dtype) # Data type of the image: uint8

image = ds_train['images'][i].numpy()  # Chuyển tensor sang NumPy array
print(image) # Numpy array
print("Shape of the image:", image.shape) # Shape of the image: (295, 285, 3)
print("Data type of the image:", image.dtype) # Data type of the image: uint8

# Hiển thị ảnh
plt.imshow(image)
plt.title(f"Image {i}")
plt.show()

# In thông tin cấu trúc của bounding box
box = ds_train['boxes'][i].numpy()  # Lấy thông tin bounding box và chuyển sang NumPy array
print(f"Box shape for image {i}:", box.shape)
print(f"First bounding box for image {i} (if multiple):", box[0])



-
```python
image = ds_train['images'][i].numpy()  # Chuyển tensor sang NumPy array
box = ds_train['boxes'][i].numpy()  # Lấy thông tin bounding box và chuyển sang NumPy array
- bb dạng: x_min, y_min, width, height

## Load Images and Bouding Boxes from deeplake to local colab
Khi bạn sử dụng `deeplake.load` để tải bộ dữ liệu từ Activeloop Hub, bộ dữ liệu được tải trực tiếp vào bộ nhớ của môi trường Python mà bạn đang sử dụng, như Google Colab hoặc Kaggle Kernel. Nó không được lưu trữ trên ổ cứng hoặc bất kỳ thiết bị lưu trữ vật lý nào, trừ khi bạn chủ động thực hiện việc lưu nó.

Dữ liệu trong `ds_train` sau khi chạy đoạn mã của bạn sẽ tồn tại dưới dạng một đối tượng trong bộ nhớ, có thể truy cập và xử lý trong phiên làm việc Python hiện tại của bạn. Khi bạn ngắt kết nối từ Colab hoặc Kaggle Kernel, hoặc khi phiên của bạn hết hạn, dữ liệu này sẽ bị mất trừ khi bạn đã lưu nó vào một nơi lưu trữ khác như Google Drive hoặc Kaggle Datasets.

Nếu bạn muốn lưu một phần hoặc toàn bộ bộ dữ liệu này vào ổ cứng hoặc một dịch vụ lưu trữ đám mây, bạn cần viết mã để thực hiện việc lưu trữ đó. Điều này thường bao gồm việc chuyển dữ liệu vào định dạng phù hợp (ví dụ, hình ảnh thành các file ảnh, dữ liệu tabular thành CSV) và sau đó sử dụng các API hoặc thư viện để lưu nó vào vị trí mong muốn.

In [ ]:
!mkdir custom_dataset
!mkdir custom_dataset/images_and_labels
!mkdir custom_dataset/images_and_labels/images
!mkdir custom_dataset/images_and_labels/labels

In [ ]:
import deeplake
import os
from PIL import Image
import numpy as np

# Load the dataset using deeplake
ds_train = deeplake.load("hub://activeloop/celeb-a-train")[:10]

# Create a directory to save images if it does not exist
save_path = "custom_dataset/images_and_labels/images/train"
os.makedirs(save_path, exist_ok=True)

# Save images to the directory
for i in range(len(ds_train['images'])):  # Assuming 'image' is the key for images
    image_array = ds_train['images'][i].numpy()  # Convert to numpy array
    if image_array.ndim == 3 and image_array.shape[2] == 3:  # Check if image is in correct format
        image = Image.fromarray(image_array)
        image_path = os.path.join(save_path, f'image_{i}.png')
        image.save(image_path, "PNG")

print("Images saved successfully.")


Để xử lý thông tin bounding box từ bộ dữ liệu CelebA và viết chúng vào các file `.txt` theo định dạng của YOLOv5, bạn cần thực hiện các bước sau:

1. **Xác Định Tọa Độ Bounding Box**: Trong YOLOv5, bounding box được biểu diễn bởi tọa độ x_center, y_center, width, và height. Tuy nhiên, tọa độ trong CelebA có thể được biểu diễn dưới dạng x_min, y_min, width, và height. Do đó, bạn cần chuyển đổi chúng.

2. **Chuyển Đổi Tọa Độ**:
   - x_center = x_min + (width / 2)
   - y_center = y_min + (height / 2)

3. **Normalize Tọa Độ**: Trong YOLOv5, tọa độ thường được chuẩn hóa theo kích thước của ảnh. Điều này có nghĩa là x_center, y_center, width, và height đều là phần trăm tương đối của kích thước ảnh.
   - x_center = x_center / chiều rộng ảnh
   - y_center = y_center / chiều cao ảnh
   - width = width / chiều rộng ảnh
   - height = height / chiều cao ảnh

4. **Lưu vào File `.txt`**: Mỗi ảnh sẽ có một file `.txt` tương ứng với nội dung là "0" và tọa độ bounding box đã được chuẩn hóa và chuyển đổi.


In [ ]:
import os

# Tạo thư mục để lưu file .txt nếu chưa tồn tại
txt_save_path = "custom_dataset/images_and_labels/labels/train"
os.makedirs(txt_save_path, exist_ok=True)

for i in range(len(ds_train['images'])):
    image = ds_train['images'][i].numpy()
    box = ds_train['boxes'][i].numpy()[0]  # Giả sử chỉ có một bounding box cho mỗi ảnh

    # Chuyển đổi tọa độ
    x_center = box[0] + (box[2] / 2)
    y_center = box[1] + (box[3] / 2)

    # Normalize tọa độ
    x_center /= image.shape[1]  # Chia cho chiều rộng ảnh
    y_center /= image.shape[0]  # Chia cho chiều cao ảnh
    width_norm = box[2] / image.shape[1]
    height_norm = box[3] / image.shape[0]

    # Tạo nội dung cho file .txt
    txt_content = f"0 {x_center} {y_center} {width_norm} {height_norm}"

    # Lưu nội dung vào file .txt
    txt_filename = f"image_{i}.txt"  # Hoặc sử dụng tên ảnh nếu có
    with open(os.path.join(txt_save_path, txt_filename), 'w') as f:
        f.write(txt_content)

print("Bounding box annotations saved successfully.")



## Processing: Train - Val - Test for Yolov5s

- https://github.com/deepakat002/yolov5_facemask
- https://www.pandaml.com/train-yolov5/
- https://www.kaggle.com/datasets/deepakat002/face-mask-detection-yolov5

```
yolov5
custom_dataset
├── custom_dataset.yaml
├── custom_model.yaml
└── images_and_labels
```

```
images_and_labels
├── images
│   ├── train
│   │   ├── train_001.jpg
│   │   ├── train_002.jpg
│   │   └── ...
│   ├── valid
│   │   ├── valid_001.jpg
│   │   ├── valid_002.jpg
│   │   └── ...
│   └── test
│       ├── test_001.jpg
│       ├── test_002.jpg
│       └── ...
└── labels
  ├── train
  │   ├── train_001.txt
  │   ├── train_002.txt
  │   └── ...
  └── valid
      ├── valid_001.txt
      ├── valid_002.txt
      └── ...
```
hoặc
```
images_and_labels
├── train
│   ├── images
│   │   ├── train_001.jpg
│   │   ├── train_002.jpg
│   │   └── ...
│    ── labels
│       ├── train_001.txt
│       ├── train_002.txt
│       └── ...                
└── val
  ├── images
  │   ├── valid_001.jpg
  │   ├── valid_002.jpg
  │   └── ...
  └── labels
      ├── valid_001.txt
      ├── valid_002.txt
      └── ...
```

In [ ]:
import deeplake
import os
from PIL import Image
import numpy as np

def process_dataset(ds, images_path, labels_path, dataset_type):
    os.makedirs(images_path, exist_ok=True)
    os.makedirs(labels_path, exist_ok=True)

    for i, (image_array, box) in enumerate(zip(ds['images'], ds['boxes'])):
        if image_array.ndim == 3 and image_array.shape[2] == 3:
            # Save image
            image = Image.fromarray(image_array.numpy())
            image_path = os.path.join(images_path, f'image_{i}.png')
            image.save(image_path, "PNG")

            # Save bounding box
            box = box.numpy()[0]
            x_center, y_center = box[0] + box[2] / 2, box[1] + box[3] / 2
            x_center /= image_array.shape[1]
            y_center /= image_array.shape[0]
            width_norm = box[2] / image_array.shape[1]
            height_norm = box[3] / image_array.shape[0]
            txt_content = f"0 {x_center} {y_center} {width_norm} {height_norm}"
            txt_filename = f"image_{i}.txt"
            with open(os.path.join(labels_path, txt_filename), 'w') as f:
                f.write(txt_content)

    print(f"{dataset_type} images and annotations saved successfully.")

# Load datasets
ds_train = deeplake.load("hub://activeloop/celeb-a-train")[:1000]
ds_val = deeplake.load("hub://activeloop/celeb-a-val")[:100]
ds_test = deeplake.load("hub://activeloop/celeb-a-test")[:100]

# Define paths
base_path = "custom_dataset/images_and_labels"
image_paths = {
    "train": os.path.join(base_path, "images/train"),
    "valid": os.path.join(base_path, "images/valid"),
    "test": os.path.join(base_path, "images/test")
}
label_paths = {
    "train": os.path.join(base_path, "labels/train"),
    "valid": os.path.join(base_path, "labels/valid"),
    "test": os.path.join(base_path, "labels/test")
}

# Process datasets
process_dataset(ds_train, image_paths['train'], label_paths['train'], "Train")
process_dataset(ds_val, image_paths['valid'], label_paths['valid'], "Validation")
process_dataset(ds_test, image_paths['test'], label_paths['test'], "Test")



# ***2. Model: Yolov5s***

- https://www.youtube.com/watch?v=12UoOlsRwh8
- https://github.com/deepakat002/yolov5_facemask

Write custom_dataset.yaml

In [ ]:
# !rm -r /kaggle/working/custom_dataset/custom_dataset.yaml

# Create and write to a file (cách tạo file trong kaggle luôn rùi)
with open('/content/drive/MyDrive/colab/custom_dataset/custom_dataset.yaml', 'w') as file:  # write ghi đè, a (append mode)
    file.write("""
# train: /kaggle/working/custom_dataset/images_and_labels/images/train
# val: /kaggle/working/custom_dataset/images_and_labels/images/valid
train: /content/drive/MyDrive/colab/custom_dataset/images_and_labels/images/train
val: /content/drive/MyDrive/colab/custom_dataset/images_and_labels/images/valid
# number of classes
nc: 1
# class names
names: ['face']
""")


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!cd yolov5 && pwd
%cd yolov5
!pip install -r requirements.txt

In [ ]:
# https://github.com/ultralytics/yolov5/releases/v6.0
# https://github.com/ultralytics/yolov5/releases/v7.0
!wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt
# !wget https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5s.pt

## Training

In [4]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00


In [5]:
import wandb
wandb.login(key = "c8767797aae76cbcd389ff29929ace1ac3021161")    # key's DoanNgocCuong
wandb.init(
    project = "FaceDetection_Yolov5"
)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin


In [3]:
%cd /content/drive/MyDrive/colab/yolov5

/content/drive/MyDrive/colab/yolov5


- Using a Pretrained Weight File
The model's weights are adjusted during the training process on your custom dataset. This approach is often used for transfer learning, where the pretrained model has already learned features from a large and diverse dataset.
- Using a Configuration File Without Pretrained Weights which defines the architecture of the model but does not use pretrained weights. The model will be trained from scratch on your custom dataset.

- cfg: cấu hình model từ đầu,
- weights để trống: ko xài pretrained model
- cache: load data vào train.cache ở RAM, tối ưu việc train.
- project (log with wandb)

In [7]:
# Train từ scratch
!python train.py --img 416 --batch 16 --epochs 10 \
  --data /content/drive/MyDrive/colab/custom_dataset/custom_dataset.yaml \
  --cfg /content/drive/MyDrive/colab/yolov5/models/yolov5s.yaml \
  --weights '' \
  --name facedet_celeba_cfgyolov5_colab --cache \
  --project Yolov5_FaceDetection


# # Train từ pretrained model: !wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt
# !python train.py --img 416 --batch 16 --epochs 10 \
#   --data /content/drive/MyDrive/colab/custom_dataset/custom_dataset.yaml \
#   --weights '/content/drive/MyDrive/colab/yolov5/yolov5s.pt' --name facedet_celeba_weightsyolov5s_colab --cache \
#   --project Yolov5_FaceDetection

# # Train từ checkpoints model.
# !python train.py --img 416 --batch 16 --epochs 10 \
#   --data /content/drive/MyDrive/colab/custom_dataset/custom_dataset.yaml \
#   --weights '/content/drive/MyDrive/colab/yolov5/Yolov5_FaceDetection/custom_model/weights/best.pt' --name checkpoint_model --cache \
#   --project Yolov5_FaceDetection

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-01-02 22:51:33.559906: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 22:51:33.559962: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 22:51:33.561554: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: doanngoccuong (doanngoccuong_nh). Use `wandb login --relogin` to force relogin
train: weights=, cfg=/content/drive/MyDrive/colab/yolov5/models/yolov5s.yaml, data=/content/dr

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

## Inference or detection on new images


### Infer1: Train 1000, Val: 100, Test: 100. Use: pretrained model: yolov5s.pt



```python
# Train từ pretrained model: !wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt
!python train.py --img 416 --batch 16 --epochs 10 \
  --data /content/drive/MyDrive/colab/custom_dataset/custom_dataset.yaml \
  --weights '/content/drive/MyDrive/colab/yolov5/yolov5s.pt' --name custom_model --cache \
  --project Yolov5_FaceDetection
```


- Khi ko connect wandb, thì sau train, thông tin train được lưu tại.

/content/drive/MyDrive/colab/yolov5/runs/train/custom_model123456
- Nhưng khi connect wandb thì ko thấy runs/train nữa, thay vào đó là:
/content/drive/MyDrive/colab/yolov5/Yolov5_FaceDetection

In [8]:
# --img 416 --save-txt --save-conf (ko cần resize 416 cũng ukiii)
# Infer trên 100 images test
!python detect.py --source /content/drive/MyDrive/colab/custom_dataset/images_and_labels/images/test --weights /content/drive/MyDrive/colab/yolov5/Yolov5_FaceDetection/facedet_celeba_weightsyolov5s_colab/weights/best.pt

detect: weights=['/content/drive/MyDrive/colab/yolov5/Yolov5_FaceDetection/custom_model/weights/best.pt'], source=/content/drive/MyDrive/colab/custom_dataset/images_and_labels/images/test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-256-g43c43d8 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Traceback (most recent call last):
  File "/content/drive/MyDrive/colab/yolov5/detect.py", line 285, in <module>
    main(opt)
  File "/content/drive/MyDrive/colab/yolov5/detect.py", line 280, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in

In [10]:
# --img 416 --save-txt --save-conf (ko cần resize 416 cũng ukiii)
!python detect.py --source /content/chidau.jpg --weights /content/drive/MyDrive/colab/yolov5/Yolov5_FaceDetection/facedet_celeba_weightsyolov5s_colab/weights/best.pt
!python detect.py --source /content/PhanNgocLanDSC01419.JPG --weights /content/drive/MyDrive/colab/yolov5/Yolov5_FaceDetection/facedet_celeba_weightsyolov5s_colab/weights/best.pt

detect: weights=['/content/drive/MyDrive/colab/yolov5/Yolov5_FaceDetection/facedet_celeba_weightsyolov5s_colab/weights/best.pt'], source=/content/drive/MyDrive/colab/custom_dataset/real_test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-256-g43c43d8 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Traceback (most recent call last):
  File "/content/drive/MyDrive/colab/yolov5/detect.py", line 285, in <module>
    main(opt)
  File "/content/drive/MyDrive/colab/yolov5/detect.py", line 280, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115,

![chidau.jpg](attachment:chidau.jpg)

![PhanNgocLanDSC01419.JPG](attachment:PhanNgocLanDSC01419.JPG)

### Infer2: Train 1000, Val: 100, Test: 100. Without pretrained model: yolov5s.pt,
use config /content/drive/MyDrive/colab/yolov5/models/yolov5s.yaml

In [12]:
# --img 416 --save-txt --save-conf (ko cần resize 416 cũng ukiii)
# # Infer trên 100 images test
# !python detect.py --source /content/drive/MyDrive/colab/custom_dataset/images_and_labels/images/test --weights /content/drive/MyDrive/colab/yolov5/Yolov5_FaceDetection/custom_model/weights/best.pt

# Infer real_test
!python detect.py --source /content/drive/MyDrive/colab/custom_dataset/real_test --weights /content/drive/MyDrive/colab/yolov5/Yolov5_FaceDetection/facedet_celeba_cfgyolov5_colab/weights/best.pt

detect: weights=['/content/drive/MyDrive/colab/yolov5/Yolov5_FaceDetection/facedet_celeba_cfgyolov5_colab/weights/best.pt'], source=/content/drive/MyDrive/colab/custom_dataset/real_test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-256-g43c43d8 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/4 /content/drive/MyDrive/colab/custom_dataset/real_test/Long.jpg: 640x640 3 faces, 11.6ms
image 2/4 /content/drive/MyDrive/colab/custom_dataset/real_test/PhanNgocLanDSC01419.JPG: 640x448 1 face, 53.5ms
image 3/4 /

- Cơ bản là độ chính xác tệ hơn khi detect ảnh Long thì thêm 2 cái ô vuông to tướng.
- Anh Phan Lan, chị Hà thì bị double boud như kiểu có 2 người. (do ảnh gốc có bounding box của lần infer trước)

mail BKE GNH